# DES Classifictaion Using Statistical Method

### Imports

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy
from astropy.table import Table, join, column, Column
import astropy
from scipy.optimize import minimize
import matplotlib.colors as colors
from sklearn.model_selection import train_test_split
from sklearn.metrics import  precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from numpy import std


### Training and Results

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# --- Load data ---
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("des_test_data2.csv")


unc_features_cols = ['g_err_mag', 'r_err_mag', 'i_err_mag', 'z_err_mag', 
'g_err_alpha', 'g_err_beta', 'r_err_alpha', 'r_err_beta',
'i_err_alpha', 'i_err_beta','z_err_alpha', 'z_err_beta']

features_cols = [
    'g_pk_mag','r_pk_mag', 'i_pk_mag', 'z_pk_mag', 
    'g_alpha', 'g_beta', 'r_alpha', 'r_beta', 'i_alpha', 'i_beta',
    'z_alpha', 'z_beta'
]
label_col = 'type'

tfeatures = df_train[features_cols].values
etfeatures = df_train[unc_features_cols].values
SNT = df_train["type"].values

# --- Preprocessing ---
invalid = (tfeatures <= -999) | (etfeatures <= 0.0001)
for k in range(tfeatures.shape[1]):
    valid_idx = ~invalid[:, k]
    if np.any(valid_idx):
        median_feat = np.median(tfeatures[valid_idx, k])
        median_unc = np.median(etfeatures[valid_idx, k]) * 240.
        tfeatures[~valid_idx, k] = median_feat
        etfeatures[~valid_idx, k] = median_unc

wa = np.where(SNT == "SN Ia")[0]
wb = np.where(SNT != "SN Ia")[0]
tfeaturesa = tfeatures[wa]
etfeaturesa = etfeatures[wa]
tfeaturesb = tfeatures[wb]
etfeaturesb = etfeatures[wb]
etfeatures[:, 0:5] = np.sqrt(etfeatures[:, 0:5]**2 + 0.05**2)

na = len(wa)
nb = len(wb)

features = df_test[features_cols].values
efeatures = df_test[unc_features_cols].values
SNTtest = df_test["type"].values
efeatures[:, 0:5] = np.sqrt(efeatures[:, 0:5]**2 + 0.05**2)

features1 = df_test[features_cols].values
efeatures1 = df_test[unc_features_cols].values

invalid_test1 = (features1 <= -999) | (efeatures1 <= 0.0001)
features1[invalid_test1] = np.nan





invalid_test = (features <= -999) | (efeatures <= 0.0001)
features[invalid_test] = 9.99
efeatures[invalid_test] = 1e13

# --- PA function ---
def PA(a, features, efeatures, tfeaturesa, etfeaturesa, na, tfeaturesb, etfeaturesb, nb):
    lnpa = np.array([
        -0.5 * (np.sum((tfeaturesa[i] - features)**2 / (etfeaturesa[i]**2 + efeatures**2)) +
                np.sum(np.log(etfeaturesa[i]**2 + efeatures**2)))
        for i in range(na)
    ])

    

    lnpb = np.array([
        -0.5 * (np.sum((tfeaturesb[i] - features)**2 / (etfeaturesb[i]**2 + efeatures**2)) +
                np.sum(np.log(etfeaturesb[i]**2 + efeatures**2)))
        for i in range(nb)
    ])
    mln = max(np.max(lnpa), np.max(lnpb))
    # lnpa -= mln
    # lnpb -= mln
    wa = lnpa > -700
    wb = lnpb > -700


    pa = np.sum(np.exp(lnpa[wa])) if np.any(wa) else 0
    pb = np.sum(np.exp(lnpb[wb])) if np.any(wb) else 0
    if not np.any(wa) and not np.any(wb):
        return 0.5
    if pa == 0 and pb == 0 or nb == 0:
        raise RuntimeError("Invalid PA computation")
    return pa / (pa + a * na / nb * pb)

# --- Optimization ---
naiter = 1
mina = 0.8
maxa = 1.2

#a_vals=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
a_vals = np.linspace(mina, maxa, naiter+1)
PASN = np.zeros((naiter+1, len(SNTtest)))
loss = np.zeros(naiter+1)

wa_test = np.where(SNTtest == 1)[0]
wb_test = np.where(SNTtest != 1)[0]

t0 = time.time()
for ia, a in enumerate(a_vals):
    for i in range(len(SNTtest)):
        PASN[ia, i] = PA(a, features[i], efeatures[i], tfeaturesa, etfeaturesa, na, tfeaturesb, etfeaturesb, nb)
    loss[ia] = np.sum(1.0 - PASN[ia, wa_test]) + np.sum(PASN[ia, wb_test])
    print(f"weight = {a:.3f}, Loss = {loss[ia]:.3f}, Time = {time.time() - t0:.2f}s")



a_loss_array = np.column_stack((a_vals, loss))

# Save only a_loss array
#np.savez("PSN0.01_aloss.npz", a_loss=a_loss_array)
df_results = pd.DataFrame(a_loss_array, columns=["a", "loss"])


wx = np.argmin(loss)




yy=len([x for x in PASN[wx, wa_test] if x > 0.5])+len([x for x in PASN[wx, wb_test] if x < 0.5])
print('Accuracy:')
print( yy/(len(PASN[wx, wa_test])+len(PASN[wx, wb_test])))



weight = 0.800, Loss = 53.189, Time = 21.16s
weight = 1.200, Loss = 53.568, Time = 42.22s
Accuracy:
0.8859649122807017
